In [6]:
import keras
from keras.layers import Conv2D, Conv2DTranspose, Input, Flatten, Dense, Lambda, Reshape
from keras.layers import BatchNormalization
from keras.models import Model
from keras.datasets import mnist
from keras.losses import binary_crossentropy
from keras import backend as K
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

Using TensorFlow backend.


In [1]:
from numpy import load
dict_data = load('/content/drive/My Drive/Mineria/datos.npz')
datos = dict_data['arr_0']
datos=datos.reshape((23705,200,200,3))
datos.shape

(23705, 200, 200, 3)

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test = train_test_split(datos, test_size=0.25, random_state=42)

In [0]:
# Data & model configuration
img_width, img_height = X_train.shape[1], X_train.shape[2]
batch_size = 64 #128
no_epochs = 100 #100
validation_split = 0.2 #20% validacion
verbosity = 1 # Se muestran los resultados
latent_dim = 2 #2 dimensiones de separacion del tensor?
num_channels = 3 #planos de color

# Reshape data
input_train = X_train.reshape(X_train.shape[0], img_height, img_width, num_channels)
input_test = X_test.reshape(X_test.shape[0], img_height, img_width, num_channels)
input_shape = (img_height, img_width, num_channels)

In [4]:
input_train = input_train.astype('float32')
input_test = input_test.astype('float32')
print("input_train.shape=",input_train.shape)
print('input_test.shape',input_test.shape)

input_train.shape= (17778, 200, 200, 3)
input_test.shape (5927, 200, 200, 3)


In [0]:
# # =================
# # Encoder
# # =================

# Definition
i       = Input(shape=input_shape, name='encoder_input')
cx      = Conv2D(filters=8, kernel_size=3, strides=2, padding='same', activation='relu')(i) #facilita la comprensión de datos
cx      = BatchNormalization()(cx) #Se asegura media 0 y varianza sigma
cx      = Conv2D(filters=16, kernel_size=3, strides=2, padding='same', activation='relu')(cx) #facilita la comprensión de datos
cx      = BatchNormalization()(cx) #Se asegura media 0 y varianza sigma
x       = Flatten()(cx) # one-dimensional shape
x       = Dense(20, activation='relu')(x) #Autoencoder?
x       = BatchNormalization()(x)
mu      = Dense(latent_dim, name='latent_mu')(x) #parametros del encoder
sigma   = Dense(latent_dim, name='latent_sigma')(x)

# Get Conv2D shape for Conv2DTranspose operation in decoder
conv_shape = K.int_shape(cx) 

In [8]:
# Define sampling with reparameterization trick
def sample_z(args):
  mu, sigma = args
  batch     = K.shape(mu)[0]
  dim       = K.int_shape(mu)[1]
  eps       = K.random_normal(shape=(batch, dim))
  return mu + K.exp(sigma / 2) * eps

# Use reparameterization trick to ....??
z       = Lambda(sample_z, output_shape=(latent_dim, ), name='z')([mu, sigma])

# Instantiate encoder
encoder = Model(i, [mu, sigma, z], name='encoder')
encoder.summary()

Model: "encoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input (InputLayer)      (None, 200, 200, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 100, 100, 8)  224         encoder_input[0][0]              
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 100, 100, 8)  32          conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 50, 50, 16)   1168        batch_normalization_1[0][0]      
____________________________________________________________________________________________

In [9]:
# =================
# Decoder
# =================
# Definition
d_i   = Input(shape=(latent_dim, ), name='decoder_input')
x     = Dense(conv_shape[1] * conv_shape[2] * conv_shape[3], activation='relu')(d_i)
x     = BatchNormalization()(x)
x     = Reshape((conv_shape[1], conv_shape[2], conv_shape[3]))(x)
cx    = Conv2DTranspose(filters=16, kernel_size=3, strides=2, padding='same', activation='relu')(x)
cx    = BatchNormalization()(cx)
cx    = Conv2DTranspose(filters=8, kernel_size=3, strides=2, padding='same',  activation='relu')(cx)
cx    = BatchNormalization()(cx)
o     = Conv2DTranspose(filters=num_channels, kernel_size=3, activation='sigmoid', padding='same', name='decoder_output')(cx)

# Instantiate decoder
decoder = Model(d_i, o, name='decoder')
decoder.summary()

Model: "decoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
decoder_input (InputLayer)   (None, 2)                 0         
_________________________________________________________________
dense_2 (Dense)              (None, 40000)             120000    
_________________________________________________________________
batch_normalization_4 (Batch (None, 40000)             160000    
_________________________________________________________________
reshape_1 (Reshape)          (None, 50, 50, 16)        0         
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 100, 100, 16)      2320      
_________________________________________________________________
batch_normalization_5 (Batch (None, 100, 100, 16)      64        
_________________________________________________________________
conv2d_transpose_2 (Conv2DTr (None, 200, 200, 8)       1160

In [10]:
# =================
# VAE as a whole
# =================

vae_outputs = decoder(encoder(i)[2])
vae         = Model(i, vae_outputs, name='vae')
vae.summary()

Model: "vae"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
encoder_input (InputLayer)   (None, 200, 200, 3)       0         
_________________________________________________________________
encoder (Model)              [(None, 2), (None, 2), (N 801672    
_________________________________________________________________
decoder (Model)              (None, 200, 200, 3)       283795    
Total params: 1,085,467
Trainable params: 1,005,331
Non-trainable params: 80,136
_________________________________________________________________


In [0]:
# Define loss
def kl_reconstruction_loss(true, pred):
  # Reconstruction loss
  reconstruction_loss = binary_crossentropy(K.flatten(true), K.flatten(pred)) * img_width * img_height
  # KL divergence loss
  kl_loss = 1 + sigma - K.square(mu) - K.exp(sigma)
  kl_loss = K.sum(kl_loss, axis=-1)
  kl_loss *= -0.5
  # Total loss = 50% rec + 50% KL divergence loss
  return K.mean(reconstruction_loss + kl_loss)

In [0]:
# Compile VAE
vae.compile(optimizer='adam', loss=kl_reconstruction_loss)

# Train autoencoder
vae.fit(input_train, input_train, epochs = no_epochs, batch_size = batch_size, validation_split = validation_split)

Train on 14222 samples, validate on 3556 samples
Epoch 1/100
14222/14222 [==============================] - 137s 10ms/step - loss: 26371.4018 - val_loss: 26203.4002
Epoch 2/100
14222/14222 [==============================] - 138s 10ms/step - loss: 24938.5667 - val_loss: 25256.9050
Epoch 3/100
14222/14222 [==============================] - 137s 10ms/step - loss: 24816.7521 - val_loss: 24787.1484
Epoch 4/100
14222/14222 [==============================] - 138s 10ms/step - loss: 24761.2235 - val_loss: 24742.1716
Epoch 5/100
14222/14222 [==============================] - 135s 9ms/step - loss: 24735.7424 - val_loss: 24717.2721
Epoch 6/100
14222/14222 [==============================] - 139s 10ms/step - loss: 24712.4341 - val_loss: 24704.0448
Epoch 7/100
14222/14222 [==============================] - 137s 10ms/step - loss: 24700.6642 - val_loss: 24685.1815
Epoch 8/100
14222/14222 [==============================] - 135s 9ms/step - loss: 24690.4934 - val_loss: 24672.9622
Epoch 9/100
14222/14222 [

In [0]:
# =================
# Results visualization
# Credits for original visualization code: https://keras.io/examples/variational_autoencoder_deconv/
# (François Chollet).
# Adapted to accomodate this VAE.
# =================
def viz_latent_space(encoder, data):
  input_data, target_data = data
  mu, _, _ = encoder.predict(input_data)
  plt.figure(figsize=(8, 10))
  plt.scatter(mu[:, 0], mu[:, 1], c=target_data)
  plt.xlabel('z - dim 1')
  plt.ylabel('z - dim 2')
  plt.colorbar()
  plt.show()

def viz_decoded(encoder, decoder, data):
  num_samples = 15
  figure = np.zeros((img_width * num_samples, img_height * num_samples, num_channels))
  grid_x = np.linspace(-4, 4, num_samples)
  grid_y = np.linspace(-4, 4, num_samples)[::-1]
  for i, yi in enumerate(grid_y):
      for j, xi in enumerate(grid_x):
                z_sample = np.array([[xi, yi]])
                x_decoded = decoder.predict(z_sample)
                digit = x_decoded[0].reshape(img_width, img_height, num_channels)
                figure[i * img_width: (i + 1) * img_width,
                  j * img_height: (j + 1) * img_height] = digit
  plt.figure(figsize=(10, 10))
  start_range = img_width // 2
  end_range = num_samples * img_width + start_range + 1
  pixel_range = np.arange(start_range, end_range, img_width)
  sample_range_x = np.round(grid_x, 1)
  sample_range_y = np.round(grid_y, 1)
  plt.xticks(pixel_range, sample_range_x)
  plt.yticks(pixel_range, sample_range_y)
  plt.xlabel('z - dim 1')
  plt.ylabel('z - dim 2')
  # matplotlib.pyplot.imshow() needs a 2D array, or a 3D array with the third dimension being of shape 3 or 4!
  # So reshape if necessary
  fig_shape = np.shape(figure)
  if fig_shape[2] == 1:
    figure = figure.reshape((fig_shape[0], fig_shape[1]))
  # Show image
  plt.imshow(figure)
  plt.show()

# Plot results
data = (input_test, target_test)
viz_latent_space(encoder, data)
viz_decoded(encoder, decoder, data)